# Acceso a Mistral-7B-Instruct (Hugging Face)

Este notebook muestra cómo descargar y usar el modelo `mistralai/Mistral-7B-Instruct-v0.2` desde Hugging Face usando `transformers`. Incluye instrucciones de instalación, autenticación y una función de utilidades para generar respuestas.

## Instalación de dependencias

Las siguientes librerías son recomendadas para cargar y ejecutar modelos grandes: `transformers`, `accelerate`. Ejecuta la celda siguiente en el notebook o en tu terminal si aún no las tienes instaladas.



In [1]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
# %pip install transformers accelerate

## Imports y comprobación de dispositivo
La siguiente celda importa dependencias clave y detecta si hay GPU disponible.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Versión de torch:", torch.__version__)
print("¿CUDA disponible?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA:", torch.cuda.device_count())
    print('Nombre del dispositivo:', torch.cuda.get_device_name(0))
else:
    print('No se detectó GPU CUDA.')

Versión de torch: 2.4.1+cu121
¿CUDA disponible?: True
Número de dispositivos CUDA: 1
Nombre del dispositivo: NVIDIA GeForce RTX 3060


## Cargar el modelo y generar texto

En la siguiente celda se carga el modelo, descargándolo desde Hugging Face.


In [4]:
import os

ruta_modelo = r"C:\llm-models\mistral-local"

if not os.path.exists(ruta_modelo):
    !huggingface-cli download mistralai/Mistral-7B-Instruct-v0.2 --local-dir "{ruta_modelo}"
else:
    print(f"La carpeta '{ruta_modelo}' ya existe. Se omite la descarga.")


La carpeta 'C:\llm-models\mistral-local' ya existe. Se omite la descarga.


In [8]:
# Modelo
MODEL = "C:\llm-models\mistral-local"
use_gpu = torch.cuda.is_available()

tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    dtype=torch.float16 if use_gpu else torch.float32, # Carga en float16 si hay GPU; en CPU usa float32
    device_map="auto" if use_gpu else None, # Asigna automáticamente a GPU/CPU
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [6]:
# Función de generación de texto
def generar_texto(prompt, max_new_tokens=20, temperature=0.7, top_p=0.9):
    # Tokenizar el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generar texto
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens, # Número máximo de tokens a generar
            temperature=temperature, # Controla la aleatoriedad
            top_p=top_p, # Núcleo de muestreo
            do_sample=True, # Habilita el muestreo
        )

    # Decodificar la salida
    texto_generado = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_generado


## Prueba rápida
Ejecuta la siguiente celda para probar la función `generar_texto`. La primera ejecución cargará el modelo y puede tardar varios minutos (dependiendo de tu conexión y hardware).

In [9]:
generar_texto("La capital de Ecuador es")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'La capital de Ecuador es Quito. Quito es una ciudad hermosa y tranquila, ubicada en la'